# Jupyter Magics for More Comprehensive Workflows

## Introduction

## What is magic?

[Magic](https://en.wikipedia.org/wiki/Magic_(programming) is ubiquitous in programming, referring broadly to commands or functionality that **abstract away complexity**, usually to improve quality of life or ease of use. Like syntactic sugar, magics aren't often strictly necessary, but they're *cool*, they *save time*, and they *make you feel powerful* as you handle relatively annoying or difficult tasks with ease.

Today I want to focus on the ability of magics to bring more of your workflow into Jupyter, which not only makes things quicker but also simpler. This is a great boon, especially when putting a Jupyter workflow in front of a less experienced user who might not be as comfortable switching between several different development tools.

* What are some reasons one might have to leave Jupyter to get something accomplished?
    * Data ingestion or filtering (sed, awk)
    * Database operations
    * Unit testing
    * Frontend development (browser dev tools)
* What is the difference in the user experience?

## Where does Jupyter magic come from?

* [Built in to the kernel](http://ipython.readthedocs.io/en/stable/interactive/magics.html#)
* [Viewers like you](http://ipython.readthedocs.io/en/stable/config/custommagics.html)

## Cell magics vs line magics

Cell magics change the way the entire cell is executed, and therefore can't be mixed with other commands.
* They usually start with `%%`
* These include magics that change the REPL used to execute the cell such as `%%sh` or `%%html`.

Line commands affect a single line of code and can be run inline.
* They usually start with `%`

Why `%`?

In [ ]:
# Line magics can be inserted anywhere
%lsmagic

In [ ]:
# Cell magics must be declared at the beginning of the cell
%%html
<hr />
# Comments often won't work like you expect

## Code execution and debugging

In [ ]:
%%time
import time
time.sleep(2)

In [ ]:
%timeit time.sleep(0.25)
%timeit time.sleep(0.5)

In [ ]:
def raiser():
    x = 10
    raise NotImplementedError()

In [ ]:
# %debug also exists
%pdb

In [ ]:
raiser()

In [ ]:
%%prun
squares = [x ** 2 for x in range(10000)]
d = dict.fromkeys(squares)
list(d.keys()).pop()

## Meta-level commands and external interactions

In [ ]:
%who int

In [ ]:
%env CONDA_PREFIX

In [ ]:
%%writefile time.py
import time

if __name__ == "__main__":
    now = time.localtime()
    print("It's Daylight Saving Time!") if now.tm_isdst else print("All out of daylight...")

In [ ]:
%pycat time.py

In [ ]:
%run time.py

In [ ]:
#%run -p time.py

In [ ]:
# Persist variables across sessions with pickle
%store now

In [ ]:
# %load time.py
import time

if __name__ == "__main__":
    now = time.localtime()
    print("It's Daylight Saving Time!") if now.tm_isdst else print("All out of daylight...")

In [ ]:
print(now)  # from previous cell
%store -r now
print(now)  # from cache

## Interacting with the underlying shell

In [ ]:
%env SHELL

In [ ]:
%%sh
pwd

In [ ]:
# A line magic option
!ls -lhr
print("---")
!wc time.py

In [ ]:
# Seamless interaction with Python variables
%timeit -n1 -r3 gitbranch = !git branch 2> /dev/null | sed -e '/^[^*]/d' -e 's/* \(.*\)/ (\1)/'
print(f"The git repo is currently on the {gitbranch[0].strip(' ()')} branch.")

In [ ]:
!git diff magic.ipynb

In [ ]:
#!rm time.py

## Interacting with other kernels

## Creating your own magics

* [Full documentation](http://ipython.readthedocs.io/en/stable/config/custommagics.html)
* [Blog post example](http://mlexplained.com/2017/12/28/creating-custom-magic-commands-in-jupyter/)

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def lmagic(line):
    "my line magic"
    return line

@register_cell_magic
def cmagic(line, cell):
    "my cell magic"
    return line, cell

@register_line_cell_magic
def lcmagic(line, cell=None):
    "Magic that works both as %lcmagic and as %%lcmagic"
    if cell is None:
        print("Called as line magic")
        return line
    else:
        print("Called as cell magic")
        return line, cell

In [ ]:
%lcmagic here
x = 10
print(x**2)

### Why use the magics class?

In [ ]:
from IPython.core.magic import (Magics, magics_class, line_cell_magic, cell_magic, line_magic)
from IPython.core import magic_arguments

@magics_class
class Abracadabra(Magics):
    @line_cell_magic
    def getvars(self, line, cell=None):
        print("Full access to the main IPython object:", self.shell)
        print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        return line, cell
        
    @cell_magic
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('--out', '-o', help='The variable to return the results in')
    def message(self, line='', cell=None):
        args = magic_arguments.parse_argstring(self.message, line)
        if args.out is None:
            print('hello ' + cell)
        else:
            self.shell.user_ns[args.out] = 'hello ' + cell
            self.shell.user_ns["varlist"] = list(self.shell.user_ns.keys())

In [ ]:
%getvars

In [ ]:
ip = get_ipython()
ip.register_magics(Abracadabra)

In [ ]:
%%message
world

In [ ]:
%%message -o target
world

In [ ]:
print(target)

In [ ]:
%page varlist

## Example 1: ihtml

[Source code](https://github.com/thedataincubator/ihtml/blob/master/README.ipynb)

* Note that there is built in iframe support

## Example 2: jupyter-slack-notify

[Source code](https://github.com/keitakurita/jupyter-slack-notify)

* Notice the cell execution and timing